# Fintech - Part 2


In [1]:
import yfinance as yf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def get_yahoo_finance_data(ticker: str, start_date: str = None, end_date: str = None):
    # Create a ticker object
    stock = yf.Ticker(ticker)
    
    # Fetch historical market data
    hist = stock.history(period="max", start=start_date, end=end_date)
    
    # Return the data
    return hist

# Example usage
ticker = '^GSPC'  # Apple Inc. ticker symbol
data = get_yahoo_finance_data(ticker)

# Display the data
data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1927-12-30 00:00:00-05:00,17.660000,17.660000,17.660000,17.660000,0,0.0,0.0
1928-01-03 00:00:00-05:00,17.760000,17.760000,17.760000,17.760000,0,0.0,0.0
1928-01-04 00:00:00-05:00,17.719999,17.719999,17.719999,17.719999,0,0.0,0.0
1928-01-05 00:00:00-05:00,17.549999,17.549999,17.549999,17.549999,0,0.0,0.0
1928-01-06 00:00:00-05:00,17.660000,17.660000,17.660000,17.660000,0,0.0,0.0
...,...,...,...,...,...,...,...
2024-07-29 00:00:00-04:00,5476.549805,5487.740234,5444.439941,5463.540039,3379970000,0.0,0.0
2024-07-30 00:00:00-04:00,5478.729980,5489.459961,5401.700195,5436.439941,3777740000,0.0,0.0
2024-07-31 00:00:00-04:00,5505.589844,5551.509766,5493.750000,5522.299805,4546910000,0.0,0.0


In [3]:
tickers = {
    "S&P500": "^GSPC",
    "Crude Oil": "CL=F",
    "Silver": "SI=F", # No
    "Iron": "TIO=F",
    "NIFTY 50": "^NSEI",
    "NYSE Composite": "^NYA",
    "Candian Dollar": "CNY=X",
    "Euro": "EUR=X",
}


In [4]:
dfs = {
    name: get_yahoo_finance_data(ticker) for name, ticker in tickers.items()
}

# Convert indices to date-only format and find the intersection of all dates
common_dates = set.intersection(*(set(df.index.date) for df in dfs.values()))

# Find the earliest common date
earliest_common_date = min(common_dates)

print(f"The earliest common date is: {earliest_common_date}")

# Filter dataframes to the common date range
dfs = {name: df.loc[str(earliest_common_date):] for name, df in dfs.items()}

The earliest common date is: 2010-10-14


**Gold price data**


In [5]:
gold_df = pd.read_excel("Gold_Prices.xlsx", sheet_name="Daily", skiprows=5,usecols=[2, 3],)\
    .rename(columns={"Unnamed: 2": "Date"}).set_index("Date").loc[str(earliest_common_date):]

gold_df

FileNotFoundError: [Errno 2] No such file or directory: 'Gold_Prices.xlsx'

In [ ]:
for name, df in dfs.items():
    df.index = df.index.date
    dfs[name] = df

We decided to concentarte on the Close price between 2010-2014.

In [ ]:
combined_df = pd.DataFrame(index=dfs["S&P500"].index)
rel_column = "Close"

for feature, df in dfs.items():
    combined_df[f"{feature}_{rel_column}"] = df['Close']
    # print(df['Close'])
combined_df.index = pd.DatetimeIndex(combined_df.index)

combined_df

We drop rows with NaN values to ensure the data is clean for statistical calculations.

In [ ]:
combined_df["gold_price"] = gold_df["USD"]
combined_df.dropna()

# The Correlation Matrix of our features' prices

In [ ]:
correlation_matrix = combined_df.corr()

# Plotting the Correlation Matrix
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

# Mean and Standard deviation

In [ ]:
combined_stats = combined_df.describe().loc[['mean', 'std']]
combined_stats

In [ ]:
combined_stats.T.plot(kind='bar', figsize=(14, 6), title='Mean and Standard Deviation of Close Prices')
plt.xlabel('Ticker')
plt.ylabel('Value')
plt.show()

# Detect outliers using IQR method
We identify outliers as data points where 'Close' prices are below Q1 - 1.5 * IQR or above Q3 + 1.5 * IQR.

In [ ]:
def detect_outliers_iqr(df):
    outliers_dict = {}
    for column in df.columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
        outliers_dict[column] = outliers
    return outliers_dict

outliers_dict = detect_outliers_iqr(combined_df)
for column, outliers in outliers_dict.items():
    print(f"\nOutliers for {column} using IQR method:")
    print(outliers)

**We can see that :**

1. S&P 500 (S&P500_Close) : 
    No outliers detected.
    

2. Oil (Crude Oil_Close):

    Outlier detected on 2020-04-20 with a negative price of -37.63 USD. This date is significant as it reflects the     unprecedented event when oil prices turned negative due to oversupply and storage issues during the COVID-19       pandemic.
    
    
3. Silver (Silver_Close):

    Outliers detected mostly between April 2011 and September 2011. This period saw a significant rise in silver       prices, peaking around April 2011, which was followed by high volatility.
    
    
4. Iron (Iron_Close):
    No outliers detected.
    
    
5. NIFTY 50 (NIFTY 50_Close):
    No outliers detected.
    
    
6. NYSE Composite (NYSE Composite_Close):
    No outliers detected.


7. Canadian Dollar (Candian Dollar_Close):
    No outliers detected.


8. Euro (Euro_Close):
    No outliers detected.


9. Gold (gold_price): 
    No outliers detected.